In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

   ## Overview of the Data:

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
  FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

-    name — the name of the country.

-    area— the country's total area (both land and water).

-    area_land — the country's land area in square kilometers.

-    area_water — the country's waterarea in square kilometers.

-    population — the country's population.

-    population_growth— the country's population growth as a percentage.

-    birth_rate — the country's birth rate, or the number of births per year per 1,000 people.

-    death_rate — the country's death rate, or the number of death per year per 1,000 people.



## Exploring Population Statistics
Let's start by calculating some summary statistics and look for any outlier countries.

In [8]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few things are interesting in the summary statistics on the previous screen:

- There's a country with a population of 0.
- There's a country with a population of 7256490011 (or more than 7.2 billion people).

What are these countries: (using subqueries to zoom in and find out more about these countries)

In [23]:
%%sql
SELECT name, population
  FROM facts
WHERE population = (
        SELECT MIN(population) 
          FROM facts)
   OR population = (
        SELECT MAX(population) 
          FROM facts);

Done.


name,population
Antarctica,0
World,7256490011


It looks like table contains Antartica which explaines why the population is 0. This seem to mat. The other issue is the data counts World as a country. 
We have to recompute the summary statistic we calculated earlier. This time excludes the row for the world and Antartica. 

In [25]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name != 'World'
   AND name != 'Antartica';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


The numbers looks more plausible now without Antartica and World's values. 

Let's calculate the average population and average area:

In [32]:
%%sql
SELECT AVG(population) AS avg_population,
       AVG(area) AS avg_area
  FROM facts
 WHERE name != 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


The average population is about 33 million and the average area is 555 thousands square kilometers.

## Finding Densely Populated Countries:
To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have the following:

Above-average values for population.
Below-average values for area.

In [37]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name !='World'
              )
 ORDER BY population DESC ;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Which country has the most people? 

In [39]:
%%sql
SELECT name, population AS top10_population
  FROM facts
 WHERE name != 'World'
   AND name != 'European Union'
 ORDER BY population DESC
 LIMIT 10;

Done.


name,top10_population
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773
Japan,126919659


Which country has the least population?

In [41]:
%%sql
SELECT name, population AS bottom10_population
  FROM facts
 WHERE name != 'Antarctica'
   AND population NOT NULL
 ORDER BY population
 LIMIT 10;

Done.


name,bottom10_population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337
Christmas Island,1530
Svalbard,1872
Norfolk Island,2210
Falkland Islands (Islas Malvinas),3361
Montserrat,5241


We also recognize some Null values in population column. Let's find out which countries these are: 

In [42]:
%%sql 
SELECT name, population
  FROM facts
 WHERE population IS NULL;

Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None


The territories listed above represent uninhabited islands or the oceans.

Which countries have higher death rate than birth rate:

In [48]:
%%sql
SELECT name, ROUND(birth_rate, 1) AS top20_natural_decrease
  FROM facts
 WHERE death_rate NOT NULL
   AND birth_rate NOT NULL
 ORDER BY top20_natural_decrease
 LIMIT 20;

Done.


name,top20_natural_decrease
Malawi,-33.2
Uganda,-33.1
Niger,-33.0
Burundi,-32.7
Mali,-32.1
Burkina Faso,-30.3
Zambia,-29.5
Ethiopia,-29.1
South Sudan,-28.7
Tanzania,-28.4
